In [1]:
import pathlib

import numpy as np
import pandas as pd

import hier
import metrics

In [2]:
exp_dirs = {
    # lr = 0.1
    3: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-3/',
    4: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-4/',
    5: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-5/',
    6: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-6/',
    7: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat/',

#     # lr = 0.01
#     3: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-3-lr-0.01/',
#     4: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-4-lr-0.01/',
#     5: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-5-lr-0.01/',
#     6: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-07-inat21-trunc/flat-depth-6-lr-0.01/',
#     7: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-03-31-inat21mini/flat_softmax-lr-0.01-b-64-wd-0.0003-ep-20',
}

In [3]:
with open('resources/hierarchy/inat21.csv') as f:
    tree, names = hier.make_hierarchy_from_edges(hier.load_edges(f))

In [4]:
subtrees = {}
node_subsets = {}
projections = {}

for depth in exp_dirs:
    if depth == 7:
        subtree = tree
        node_subset = np.arange(tree.num_nodes())
        projection = np.arange(tree.num_nodes())
    else:
        subtree_file = f'resources/subtree/inat21_max_depth_{depth}.csv'
        with open(subtree_file) as f:
            subtree, subtree_names = hier.make_hierarchy_from_edges(hier.load_edges(f))
        node_subset = hier.find_subset_index(names, subtree_names)
        projection = hier.find_projection(tree, node_subset)

    subtrees[depth] = subtree
    node_subsets[depth] = node_subset
    projections[depth] = projection

In [5]:
acc = {}

for train_depth in exp_dirs:
    output_file = pathlib.Path(exp_dirs[train_depth]) / 'predictions/output-epoch-0020.pkl'
    outputs = np.load(output_file, allow_pickle=True)

    # To use 'leaf' predictions, we should load the full probability vector
    # and do max-leaf inference within the sub-tree.
    # On the other hand, 'majority' inference can be projected post-hoc.
    pred_subtree = outputs['pred']['majority']
    gt_subtree = outputs['gt']
    train_node_subset = node_subsets[train_depth]
    pred = train_node_subset[pred_subtree]
    gt = train_node_subset[gt_subtree]

    for test_depth in exp_dirs:
        if test_depth > train_depth:
            continue
        # Project ground-truth to test
        project = projections[test_depth]
        gt_proj = project[gt]
        pred_proj = project[pred]
        acc[train_depth, test_depth] = np.mean(gt_proj == pred_proj)

In [6]:
pd.Series(acc).unstack()

,3,4,5,6,7
3,0.78844,NaN,NaN,NaN,NaN
4,0.81713,0.48301,NaN,NaN,NaN
5,0.83108,0.52361,0.35589,NaN,NaN
6,0.83270,0.55167,0.40487,0.26763,NaN
7,0.83447,0.57058,0.42960,0.30600,0.22784


In [7]:
# Check whether any classes are much better or worse.

per_class = {}

for train_depth in exp_dirs:
    output_file = pathlib.Path(exp_dirs[train_depth]) / 'predictions/output-epoch-0020.pkl'
    outputs = np.load(output_file, allow_pickle=True)

    # To use 'leaf' predictions, we should load the full probability vector
    # and do max-leaf inference within the sub-tree.
    # On the other hand, 'majority' inference can be projected post-hoc.
    pred_subtree = outputs['pred']['majority']
    gt_subtree = outputs['gt']
    train_node_subset = node_subsets[train_depth]
    pred = train_node_subset[pred_subtree]
    gt = train_node_subset[gt_subtree]

    for test_depth in exp_dirs:
        if test_depth > train_depth:
            continue
        # Project ground-truth to test
        project = projections[test_depth]
        gt_proj = project[gt]
        pred_proj = project[pred]

        is_correct = (gt_proj == pred_proj)
        per_class[train_depth, test_depth] = np.array([
            np.mean(is_correct[gt_proj == y])
            for y in subtrees[test_depth].leaf_subset()
        ])

In [8]:
rows = {}

for test_depth in exp_dirs:
    for depth_a in exp_dirs:
        if depth_a < test_depth:
            continue
        for depth_b in exp_dirs:
            if depth_b <= depth_a:
                continue
            rows[test_depth, depth_a, depth_b] = pd.DataFrame(
                [per_class[depth_a, test_depth] - per_class[depth_b, test_depth]]
            ).T.describe([0.5, 0.85, 0.95]).T

table = pd.concat(rows)
table = table.droplevel(3)
table.index = table.index.set_names(['test', 'a', 'b'])
del table['count']
# table.format('{:+.1%}')

def highlight(s):
    if s.name == 'std':
        return None
    return ['color: green;' if x > 0 else 'color:black' for x in s]

table.style.apply(highlight).format('{:+.1%}')